In [73]:
# Import Libraries
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import copy
from queue import PriorityQueue
import seaborn as sns

## Ship Layout Generation

In [74]:
# Function to check open neighbors around a cell
def check_open_neighbours(ship, D, row, col):
    result = []
    if row - 1 >= 0 and ship[row - 1][col] in ('1','C','B'):
        result.append((row-1,col))
    if row + 1 < D and ship[row + 1][col] in ('1','C','B'):
        result.append((row+1,col))
    if col - 1 >= 0 and ship[row][col - 1] in ('1','C','B'):
        result.append((row,col-1))
    if col + 1 < D and ship[row][col + 1] in ('1','C','B'):
        result.append((row,col+1))
    return result

# Function to check blocked neighbors around a cell
def check_blocked_neighbours(ship, D, row, col):
    result = []
    if row - 1 >= 0 and ship[row - 1][col] == "0":
        result.append((row-1,col))
    if row + 1 < D and ship[row + 1][col] == "0":
        result.append((row+1,col))
    if col - 1 >= 0 and ship[row][col - 1] == "0":
        result.append((row,col-1))
    if col + 1 < D and ship[row][col + 1] == "0":
        result.append((row,col+1))
    return result

# Function to get neighbours around the aliens
def get_alien_neighbours(ship, D, aliens_pos):
    result = []
    for alien in aliens_pos:
            x,y = alien
            if(x-1 >= 0 ):
                result.append((x-1,y))
            if(x+1 < D ):
                result.append((x+1,y))
            if(y-1 >= 0 ):
                result.append((x,y-1))
            if(y+1 < D ):
                result.append((x,y+1))
    return result

# Function to get blocked cells with only one open neighbor
def get_blocked_cells_with_one_open_neighbor(ship, D):
    result = []
    for i in range(D):
        for j in range(D):
            if ship[i][j] == '0':
                neighbours = check_open_neighbours(ship, D, i, j)
                if len(neighbours) == 1:
                    result.append((i, j))
    return result

# Function to get open cells in the ship
def get_open_cells(ship, D):
    result = []
    for i in range(D):
        for j in range(D):
            if ship[i][j] in ('1','C'):
                result.append((i, j))
    return result

# Function to get dead ends in the ship
def get_dead_ends(ship, D):
    result = []
    for i in range(D):
        for j in range(D):
            if ship[i][j] == '1':
                neighbours = check_open_neighbours(ship, D, i, j)
                if len(neighbours) == 1:
                    result.append((i, j))
    return result

# Function to get open neighbours
def get_open_neighbours(ship, D, row, col):
    result = []
    if row - 1 >= 0 and ship[row - 1][col] not in ('0'):
        result.append((row-1,col))
    if row + 1 < D and ship[row + 1][col] not in ('0'):
        result.append((row+1,col))
    if col - 1 >= 0 and ship[row][col - 1] not in ('0'):
        result.append((row,col-1))
    if col + 1 < D and ship[row][col + 1] not in ('0'):
        result.append((row,col+1))
    return result

# Function to generate a random ship layout
def generate_ship_layout(D):
    ship = [['0' for _ in range(D)] for _ in range(D)]
    random.seed()

    # Randomly unlocking a cell on the ship
    start_row = random.randint(0, D - 1)
    start_col = random.randint(0, D - 1)
    ship[start_row][start_col] = '1'

    while True:

        blocked_cells = get_blocked_cells_with_one_open_neighbor(ship, D)
        if not blocked_cells:
            break
        index = random.randint(0, len(blocked_cells) - 1)
        new_x, new_y = blocked_cells[index]
        ship[new_x][new_y] = '1'

    dead_ends = get_dead_ends(ship, D)
    random.seed()

    # Opening the closed neighbors of approximately half of the dead-end cells at random.
    for _ in range(len(dead_ends) // 2):
        index = random.randint(0, len(dead_ends) - 1)
        new_x, new_y = dead_ends[index]
        if ship[new_x][new_y] == '1':
            blockedNeighbours = check_blocked_neighbours(ship, D, new_x, new_y)
            if len(blockedNeighbours) >= 1:
                index = random.randint(0, len(blockedNeighbours) - 1)
                new_x, new_y = blockedNeighbours[index]
                ship[new_x][new_y] = '1'

    return ship

## Alien Detection Square

In [75]:
# Function to get alien detection cells ((2k+1) * (2k+1) cells)
def get_sensor_cells(ship, D, bot_position, k):
    sensor_cells = []
    x, y = bot_position

    for i in range(x - k, x + k + 1):
        for j in range(y - k, y + k + 1):
            if 0 <= i < D and 0 <= j < D:
                if (i,j) != bot_position and ship[i][j] !='0':
                    sensor_cells.append((i, j))
    return sensor_cells
    

## Heuristic & A-star Algorithm

In [76]:
# Function to optimise the bot path using alien and crew probabilities
def heuristic_bot8(ship, D, a, b, alien_probability, crew_probability, k):
    heuristic = 0
    manhattan_dist = abs(a[0] - b[0]) + abs(a[1] - b[1])
    alien_prob = calculate_density(ship, D, a, alien_probability, k)
    crew_prob = crew_probability[a[0]][a[1]]
    heuristic = 0.8 * manhattan_dist + 0.4 * alien_prob - 0.8 * crew_prob
    return heuristic

# Function to add alien probabilities in the alien detection square
def calculate_density(ship, D, a, alien_probability, k):
    region = get_sensor_cells(ship, D, a, k)
    sum = 0
    for cell in region:
        sum += alien_probability[cell[0]][cell[1]]
    return sum


# A-star algorithm to find the path from start to goal
def get_bot8_path_a_star(ship, D, start, goal, alien_probability, crew_probability, k):
    fringe = PriorityQueue()
    fringe.put((0,start))
    dist = { start:0 }
    prev = {}
    while not fringe.empty():
        _, curr = fringe.get()
        if curr == goal :
            path = []
            while curr in prev:
                path.append(curr)
                curr = prev[curr]
            path.append(start)
            return path[::-1]

        x,y = curr
        neighbors = check_open_neighbours(ship, D, x, y)

        for neighbor in neighbors:
            tempDist = dist[curr] + 1
            if neighbor not in dist or tempDist < dist[neighbor] :
                dist[neighbor] = tempDist
                prev[neighbor] = curr
                priority = dist[neighbor] + heuristic_bot8(ship, D, neighbor, goal, alien_probability, crew_probability, k)
                fringe.put((priority,neighbor))
    return None


## Initiating bot, crew, aliens

In [77]:
# Function to get the bot position
def initiate_bot(ship, D):
    open_cells = get_open_cells(ship, D)
    index = random.randint(0, len(open_cells) - 1)
    b_x, b_y = open_cells[index]
    ship[b_x][b_y] = 'B'
    start = (b_x,b_y)
    return start

# Function to get the crew position
def initiate_crew(ship, D):
    open_cells = get_open_cells(ship, D)
    while True:
        index = random.randint(0, len(open_cells) - 1)
        c_x, c_y = open_cells[index]
        if ship[c_x][c_y] not in ('B','C') :
            ship[c_x][c_y] = 'C'
            break
    goal = (c_x,c_y)
    return goal

# Function to get the alien position
def initiate_alien(ship, D, sensor_cells):
    open_cells = get_open_cells(ship, D)
    while True:
        index = random.randint(0, len(open_cells) - 1)
        new_x, new_y = open_cells[index]
        if ship[new_x][new_y] != 'B':
            if (new_x, new_y) not in sensor_cells:
                ship[new_x][new_y] = 'A'
                return (new_x,new_y)

## Crew and Alien Beep Simulation

In [78]:
# Function to calculate manhattan distance between the two points
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# Function to calculate manhattan distance from every cell of the ship
def get_pre_computed_distances(D):
    distances = {}
    for i in range(D):
        for j in range(D):
            start = (i,j)
            for k in range(D):
                for l in range(D):
                    dist = heuristic(start,(k,l))
                    distances[(start,(k,l))] = dist
    return distances

# Function to get the alien beep
def alien_beep(aliens_pos, sensor_cells):
    if (aliens_pos[0] in sensor_cells):
        return True
    return False

# Function to get the crew beep
def crew_beep(curr, crew_pos, pre_compute_distance, alpha):
    x,y = curr
    beep2 = 0
    d = pre_compute_distance[(crew_pos[0],curr)]
    beep1 = math.exp(-alpha * (d - 1))
    if(len(crew_pos)>1):
        d = pre_compute_distance[(crew_pos[1],curr)]
        beep2 = math.exp(-alpha * (d - 1))
    index = random.random()
    if index < beep1 or index < beep2:
        return True
    else:
        return False

## Moving Aliens and Checking Game Status

In [79]:
# Function to move all aliens 1 step either up/down/right/left
def move_aliens(ship,D, aliens_pos):
    new_alien_pos = []
    
    for alien in aliens_pos:
        x,y = alien
        next_move=alien
        neighbors = check_open_neighbours(ship, D, x, y)

        # Alien moves only if it has any open cells as neighbors
        next_move  = random.choice(neighbors) if neighbors else (x, y)
        new_x,new_y = next_move
        ship[new_x][new_y] = 'A'
        ship[x][y] = '1'
        new_alien_pos.append(next_move)
    return new_alien_pos

# Function to check the game status after the bot moves
def check_after_bot_moves(ship, D, crew_pos, bot_pos, aliens_pos):

    # If bot reached goal and alien is not present at goal
    if(bot_pos in crew_pos and bot_pos not in aliens_pos):
        return "SUCCESS",aliens_pos

    # If bot reached a cell where alien is present
    elif(bot_pos in aliens_pos):
        return "FAILURE",bot_pos

    # Move the aliens to a random neighboring cell
    aliens_pos = move_aliens(ship, D, aliens_pos)

    # If aliens reach the bot
    if bot_pos in aliens_pos:
        return "FAILURE",bot_pos

    return "NEXT",aliens_pos

## Updating Probabilities

### Updating Crew Probabilities

In [80]:
# Function to update joint crew probabilities depending on beep
def update_joint_crew_probabilities(ship, joint_crew_probability, crew_probability, crew_detected, D, bot_pos, pre_computed_distance, crew_pos, alpha):

    x,y = bot_pos
    updated_crew_probability = np.zeros_like(crew_probability)

    # Calculate the denominator
    sum = 0
    for i in range(D):
        for j in range(D):
            if joint_crew_probability[i][j] != 0:
                for k in range(D):
                    for l in range(D):
                        prob_crew1 =math.exp(-alpha * (pre_computed_distance[bot_pos,(i,j)] - 1))
                        prob_crew2 = 0
                        if len(crew_pos) > 1:
                            prob_crew2 = math.exp(-alpha * (pre_computed_distance[bot_pos,(k,l)] - 1))
                        beep_given_two_crew = prob_crew1 + prob_crew2 - (prob_crew1 * prob_crew2)
                        if crew_detected:
                            sum += crew_probability[i][j][k][l] *  beep_given_two_crew
                        else:
                            sum += crew_probability[i][j][k][l] *  (1-beep_given_two_crew)


    prob_of_beep_or_no_beep=sum
    max_sum = 0

    # Probability of beep/no beep when crews are in cells (i,j) and (k,l)
    for i in range(D):
        for j in range(D):
            beep_given_two_crew = 0
            if joint_crew_probability[i][j] != 0:
                sum = 0
                for k in range(D):
                    for l in range(D):
                        prob_crew1 =math.exp(-alpha * (pre_computed_distance[bot_pos,(i,j)] - 1))
                        prob_crew2 = 0
                        if len(crew_pos) > 1:
                            prob_crew2 = math.exp(-alpha * (pre_computed_distance[bot_pos,(k,l)] - 1))
                        beep_given_two_crew = prob_crew1 + prob_crew2 - (prob_crew1 * prob_crew2)
                        if crew_detected:
                            updated_crew_probability[i][j][k][l] = crew_probability[i][j][k][l] * beep_given_two_crew / prob_of_beep_or_no_beep
                        else:
                            updated_crew_probability[i][j][k][l] = crew_probability[i][j][k][l] * (1-beep_given_two_crew) / prob_of_beep_or_no_beep
                        sum += updated_crew_probability[i][j][k][l]

                joint_crew_probability[i][j] = sum
                if sum > max_sum and ship[i][j] != 'B':
                    goal = (i, j)
                    max_sum = sum

    return updated_crew_probability, goal
    
    
# Normalization
def normalize_joint_prob_after_bot_move(D, crew_probability, joint_crew_probability, alien_probability, joint_alien_probability, bot_pos):
    x,y = bot_pos
    
    denominator1 = 1 - joint_crew_probability[bot_pos]
    joint_crew_probability[bot_pos] = 0
    for i in range(D):
        for j in range(D):
            denominator1 -= crew_probability[i][j][bot_pos[0]][bot_pos[1]]
            crew_probability[i][j][bot_pos[0]][bot_pos[1]] = 0

    for i in range(D):
        for j in range(D):
            if joint_crew_probability[i][j] != 0:
                sums = 0
                for p in range(D):
                    for q in range(D):
                        if crew_probability[i][j][p][q] != 0:
                            crew_probability[i][j][p][q] = (crew_probability[i][j][p][q] / denominator1)
                            sums += crew_probability[i][j][p][q]

                joint_crew_probability[i][j] = sums
                
    denominator2 = 1 - joint_alien_probability[bot_pos]
    joint_alien_probability[bot_pos] = 0
    for i in range(D):
        for j in range(D):
            denominator2 -= alien_probability[i][j][bot_pos[0]][bot_pos[1]]
            alien_probability[i][j][bot_pos[0]][bot_pos[1]] = 0

    for i in range(D):
        for j in range(D):
            if joint_alien_probability[i][j] != 0:
                sums = 0
                for p in range(D):
                    for q in range(D):
                        if alien_probability[i][j][p][q] != 0:
                            alien_probability[i][j][p][q] = (alien_probability[i][j][p][q] / denominator2)
                            sums += alien_probability[i][j][p][q]

                joint_alien_probability[i][j] = sums
    
    crew_probability[x][y] = 0
    alien_probability[x][y] = 0

### Updating Alien Probabilities

In [81]:

# Function to update joint alien probabilities depending on beep
def update_joint_alien_probabilities(alien_detected, joint_alien_probability, alien_probability, D, sensor_cells):

    # Calculate the denominator
    sum = 0
    for i in range(D):
        for j in range(D):
            if joint_alien_probability[i][j] != 0:
                for p in range(D):
                    for q in range(D):
                        if alien_probability[i][j][p][q] != 0:
                            
                            alien_pos = []
                            alien_pos.append((i,j))
                            beep1 = alien_beep(alien_pos,sensor_cells)
                            if(beep1): prob_alien1 = 1
                            else: prob_alien1 = 0
                
                            alien_pos = []
                            alien_pos.append((p,q))
                            prob_alien2 = alien_beep(alien_pos,sensor_cells)
                            beep2 = alien_beep(alien_pos,sensor_cells)
                            
                            if(beep2): prob_alien2 = 1
                            else: prob_alien2 = 0
                            
                            if(alien_detected):
                                prob = prob_alien1 + prob_alien2 - prob_alien1 * prob_alien2
                            else:
                                prob = prob_alien1 * prob_alien2
                  
                            sum += (alien_probability[i][j][p][q] * prob)
    
    
    # Probability of beep/no beep when aliens are in cells (i,j) and (p,q)
    prob_beep_or_no_beep = sum
    updated_alien_probability = copy.deepcopy(alien_probability) 


    for i in range(D):
        for j in range(D):  
            if joint_alien_probability[i][j] != 0 :
                sum = 0
                for p in range(D):
                    for q in range(D):
                        if alien_probability[i][j][p][q] != 0:
                            
                            alien_pos = []
                            alien_pos.append((i,j))
                            beep1 = alien_beep(alien_pos,sensor_cells)
                            if(beep1): prob_alien1 = 1
                            else: prob_alien1 = 0
                    
                            alien_pos = []
                            alien_pos.append((p,q))
                            prob_alien2 = alien_beep(alien_pos,sensor_cells)
                            beep2 = alien_beep(alien_pos,sensor_cells)
                            
                            if(beep2): prob_alien2 = 1
                            else: prob_alien2 = 0
                            
                            if(alien_detected):
                                prob = prob_alien1 + prob_alien2 - prob_alien1 * prob_alien2
                            else:
                                prob =  prob_alien1 * prob_alien2

                            updated_alien_probability[i][j][p][q] = float((alien_probability[i][j][p][q] * prob) / prob_beep_or_no_beep)     
                            sum += updated_alien_probability[i][j][p][q]  
                joint_alien_probability[i][j] = sum
                
# Function to update alien probabilities after movement
def update_joint_alien_probabilities_after_movement(ship, D, alien_probability, joint_alien_probability):
    temp_probability = np.zeros((D, D, D, D), dtype=np.longdouble)
    
    for i in range(D):
        for j in range(D):
            if (ship[i][j] not in ('0')):
                
                neighbours1 = get_open_neighbours(ship, D, i ,j)
                n1 = len(neighbours1)
                if n1 > 0:
                    stay_value1 = joint_alien_probability[i][j] / n1
                    for p in range(D):
                        for q in range(D):
                        
                            if (ship[p][q] not in ('0') and temp_probability[i][j][p][q] == 0):
                                
                                neighbours2 = get_open_neighbours(ship, D, p ,q)
                                n2 = len(neighbours2)
                                if(n2 > 0):
                                    stay_value2 = joint_alien_probability[p][q] / n2
                                    move_value = (stay_value1 * stay_value2)
                                    
                                    temp_probability[i][j][p][q] += move_value
                                    temp_probability[p][q][i][j] += move_value
    for i in range(D):
        for j in range(D):
            joint_alien_probability[i][j] = np.sum(alien_probability[i,j])
                        
    return alien_probability, joint_alien_probability


## Heat Map

In [82]:
'''def heatmap(status,alien_probabilities, crew_probabilities,bot_pos, aliens_pos, crew_pos):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))

    ax = sns.heatmap(alien_probabilities, fmt="d", linewidths=1, linecolor='white', ax=axs[0])
    axs[0].set_title('alien')
    # ax = sns.heatmap(data, annot= True)
    ax.plot([bot_pos[1] + 0.5], [bot_pos[0] + 0.5], marker='o', markersize=10, markeredgewidth=1, markeredgecolor='w',
            markerfacecolor='b')
    for alien_cell in aliens_pos:
        ax.plot([alien_cell[1] + 0.5], [alien_cell[0] + 0.5], marker='s', markersize=10, markeredgewidth=1,
                markeredgecolor='w', markerfacecolor='r')
    for crew_cell in crew_pos:
        ax.plot([crew_cell[1] + 0.5], [crew_cell[0] + 0.5], marker='o', markersize=8, markeredgewidth=1,
                markeredgecolor='w', markerfacecolor='g')

    ax = sns.heatmap(crew_probabilities, fmt="d", linewidths=1, linecolor='white', ax=axs[1])
    axs[1].set_title('crew')
    # ax = sns.heatmap(data, annot= True)
    ax.plot([bot_pos[1] + 0.5], [bot_pos[0] + 0.5], marker='o', markersize=10, markeredgewidth=1, markeredgecolor='w',
            markerfacecolor='b')
    for alien_cell in aliens_pos:
        ax.plot([alien_cell[1] + 0.5], [alien_cell[0] + 0.5], marker='s', markersize=10, markeredgewidth=1,
                markeredgecolor='w', markerfacecolor='r')
    for crew_cell in crew_pos:
        ax.plot([crew_cell[1] + 0.5], [crew_cell[0] + 0.5], marker='o', markersize=8, markeredgewidth=1,
                markeredgecolor='w', markerfacecolor='g')
    plt.suptitle(status)
    plt.tight_layout()
    plt.show()'''

'def heatmap(status,alien_probabilities, crew_probabilities,bot_pos, aliens_pos, crew_pos):\n    fig, axs = plt.subplots(1, 2, figsize=(10, 4))\n\n    ax = sns.heatmap(alien_probabilities, fmt="d", linewidths=1, linecolor=\'white\', ax=axs[0])\n    axs[0].set_title(\'alien\')\n    # ax = sns.heatmap(data, annot= True)\n    ax.plot([bot_pos[1] + 0.5], [bot_pos[0] + 0.5], marker=\'o\', markersize=10, markeredgewidth=1, markeredgecolor=\'w\',\n            markerfacecolor=\'b\')\n    for alien_cell in aliens_pos:\n        ax.plot([alien_cell[1] + 0.5], [alien_cell[0] + 0.5], marker=\'s\', markersize=10, markeredgewidth=1,\n                markeredgecolor=\'w\', markerfacecolor=\'r\')\n    for crew_cell in crew_pos:\n        ax.plot([crew_cell[1] + 0.5], [crew_cell[0] + 0.5], marker=\'o\', markersize=8, markeredgewidth=1,\n                markeredgecolor=\'w\', markerfacecolor=\'g\')\n\n    ax = sns.heatmap(crew_probabilities, fmt="d", linewidths=1, linecolor=\'white\', ax=axs[1])\n    axs[

## BOT 7 Simulation

In [83]:
# Function to simulate BOT - 8
def simulate_bot8(ship, D, K, start, crew_pos, pre_computed_distances, aliens_pos, alpha):

    step = 1
    crew_saved = 0
    open_cells = get_open_cells(ship, D)

    crew_probability = np.zeros((D, D, D, D), dtype=np.longdouble) 
    joint_crew_probability = np.zeros((D, D))
    alien_probability = np.zeros((D, D, D, D)) 
    joint_alien_probability = np.zeros((D, D))
    sensor_cells = get_sensor_cells(ship, D, start, K)

    # Initiate crew and alien probabilities
    for i in range(D):
        for j in range(D):
            if ship[i][j] not in ('B','0'):
                sums = 0
                for k in range(D):
                    for l in range(D):
                        if ship[k][l] not in ('B','0'):
                            if (i, j) != (k, l):
                                crew_probability[i][j][k][l] = 1 / ((len(open_cells)) * (len(open_cells) - 1))
                                sums += crew_probability[i][j][k][l]
                joint_crew_probability[i][j] = sums


    for i in range(D):
        for j in range(D):
            if ship[i][j] not in ('B','0'):
                sums = 0
                for p in range(D):
                    for q in range(D):
                        if ship[p][q] not in ('B','0'):
                            if (i, j) != (p, q):
                                alien_probability[i][j][p][q] = 1 / ((len(open_cells)) * (len(open_cells) - 1))
                                sums += alien_probability[i][j][p][q]
                joint_alien_probability[i][j] = sums
    
    bot_path = None
    final_bot_path = [start]
    crew_saved = 0

    while step<1000:

        sensor_cells = get_sensor_cells(ship, D, start, K)
    # Sense the Beeps and update prob

        # Sense a beep from crew
        crew_detected = crew_beep(start, crew_pos, pre_computed_distances, alpha)
        
        # Update joint crew probabilities depending on beep
        crew_probability, max_crew_prob = update_joint_crew_probabilities(ship, joint_crew_probability, crew_probability, crew_detected, D, start, pre_computed_distances, crew_pos, alpha)
        
        # Sense a beep from alien
        alien_detected = alien_beep(aliens_pos,sensor_cells)

        # Update joint alien probabilities depending on beep
        update_joint_alien_probabilities(alien_detected, joint_alien_probability, alien_probability, D, sensor_cells)
        
    # Plan a path for the bot

        # Compute the possible crew and alien position
        alien_max1=alien_probability[0][0][0][0]
        alien_max2=joint_alien_probability[0][0]
        (alien1_x,alien1_y) = (0,0)
        (alien2_x,alien2_y) = (0,0)
        
        for i in range(D):
            for j in range(D):
                    if joint_alien_probability[i][j] > alien_max2:
                        alien_max2=joint_alien_probability[i][j]
                        alien1_x = i
                        alien1_y = j
                    
        for p in range(D):
            for q in range(D):
                if alien_probability[alien1_x][alien1_y][p][q] > alien_max1:
                    alien_max1=alien_probability[alien1_x][alien1_y][p][q]
                    alien2_x = p
                    alien2_y = q
                
        ship_with_alien = copy.deepcopy(ship)
        ship_with_alien[alien1_x][alien1_y] = 'A'
        ship_with_alien[alien2_x][alien2_y] = 'A'
      
        # Find the path to the crew
        bot_path = get_bot8_path_a_star(ship_with_alien, D, start , max_crew_prob, joint_alien_probability, joint_crew_probability, K)

        b_x,b_y = start
        new_x,new_y = start
        if bot_path and len(bot_path) > 1:

            # Move the bot
            new_x,new_y = bot_path[1]
            ship[b_x][b_y] = '1'
            ship[new_x][new_y] = 'B'
            final_bot_path.append((new_x,new_y))

        start = (new_x,new_y)

        # Check if bot reached the captain or alien and then move the alien
        status,aliens_pos = check_after_bot_moves(ship, D, crew_pos, (new_x,new_y), aliens_pos)
    
        # Normalization
        normalize_joint_prob_after_bot_move(D, crew_probability, joint_crew_probability, alien_probability, joint_alien_probability, start)
        
        step+=1 
        if status == "NEXT":
            start = (new_x,new_y)

            # Update probabilities after alien movement
            alien_probability, joint_alien_probability = update_joint_alien_probabilities_after_movement(ship, D, alien_probability, joint_alien_probability)
            
            #heatmap("After Alien movement",alien_probability, joint_crew_probability,start, aliens_pos, crew_pos)
            continue
        
        elif status in ("SUCCESS","FAILURE"):
            
            crew_saved += 1
            if(status =="SUCCESS"):
                if crew_saved == 2:
                    return status, step
                elif crew_saved == 1:
                    if start in crew_pos:
                        crew_pos.remove(start)
                        continue
            else:
                return status, step

    return "FAILURE", step


## Main Method

In [84]:
def main():

    D = 35
    k = 8            # The size of the alien detection square
    alpha = 0.095    # The value of alpha

    # Function to generate ship
    ship = generate_ship_layout(D)

    # Pre-compute distances from every cell of the ship
    pre_computed_distances = get_pre_computed_distances(D)

    # Initiate the position for the bot
    start = initiate_bot(ship, D)

    # Get the alien detection square ((2k+1)*(2k+1) cells)
    sensor_cells = get_sensor_cells(ship, D, start, k)

    # Placing the captain randomly other than bot position in an open cell
    crew_pos = []
    goal = initiate_crew(ship, D)
    crew_pos.append(goal)
    goal = initiate_crew(ship, D)
    crew_pos.append(goal)
    
    # Placing the alien randomly other than bot position and sensor cells
    aliens_pos = []
    alien_pos = initiate_alien(ship, D, sensor_cells)
    aliens_pos.append(alien_pos)
    alien_pos = initiate_alien(ship, D, sensor_cells)
    aliens_pos.append(alien_pos)
     
    # Simulate BOT - 7
    status_bot8, steps_bot8 = simulate_bot8(ship, D, k, start, crew_pos, pre_computed_distances, aliens_pos, alpha)
    print("Bot 8-",status_bot8,steps_bot8)
                
if __name__ == "__main__":
    main()


Bot 8- SUCCESS 51
